**Part 1** — Data Loading & Exploration

In [4]:
#Import required libraries and start SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Loan Dataset Analysis") \
    .getOrCreate()

In [5]:
#Load the dataset (loan.csv) with headers
df = spark.read.csv("loan.csv", header=True, inferSchema=True)

In [9]:
#Display the first 10 rows
print("First 10 rows of the dataset:")
df.show(10, truncate=False)

First 10 rows of the dataset:
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|Occupation       |Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|IB14001    |30 |MALE  |BANK MANAGER     |SINGLE        |4          |50000 |22199      |6            |HOUSING      | 10,00,000 |5      |42,898      |6               |9                 |
|IB14008    |44 |MALE  |PROFESSOR        |MARRIED       |6          |51000 |19999      |4            |SHOPPING     |50,000     |3      |33,999      |1               |5                 |
|IB14012    |30 |FEMALE|DENTIST         

In [10]:
 #Show the schema
print("Schema of the DataFrame:")
df.printSchema()


Schema of the DataFrame:
root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



In [11]:
#Count total number of records
total_records = df.count()
print(f"Total number of records: {total_records}")

Total number of records: 500


**Part 2** — Data Cleaning

In [12]:

# Remove duplicate rows
df_clean = df.dropDuplicates()

In [14]:
# Drop rows with missing 'loan_status' or 'loan_amnt'
df_clean = df_clean.dropna(subset=["Loan Category", "Loan Amount"])


In [20]:
from pyspark.sql.functions import col

# Convert numeric columns to correct data type (loan_amnt → integer)
df_clean = df_clean.withColumn("Loan Amount", col("Loan Amount").cast("integer"))

In [21]:
# Verify cleaning
print("After cleaning:")
df_clean.show(10, truncate=False)

After cleaning:
+-----------+---+------+---------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|Occupation           |Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category     |Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+---------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|IB14804    |30 |MALE  |BUSINESS             |SINGLE        |2          |53957 |34709      |7            |GOLD LOAN         |NULL       |8      |37815       |5               |8                 |
|IB14697    |39 |FEMALE|NAVY                 |SINGLE        |4          |77699 |12482      |3            |AUTOMOBILE        |NULL       |9      |57206       |2               |8                 |
|IB14898 

In [22]:
# Check schema after type conversion
df_clean.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: integer (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)



In [23]:
# Count records after cleaning
print(f"Total records after cleaning: {df_clean.count()}")

Total records after cleaning: 500


**Part 3** — Data Analysis

In [29]:
from pyspark.sql.functions import avg, count

# Average loan amount for each loan category
avg_loan_by_category = df_clean.groupBy("Loan Category") \
    .agg(avg(col("Loan Amount")).alias("avg_loan_amount")) \
    .orderBy("avg_loan_amount", ascending=False)

print("Average Loan Amount by Loan Category:")
avg_loan_by_category.show(truncate=False)

Average Loan Amount by Loan Category:
+------------------+---------------+
|Loan Category     |avg_loan_amount|
+------------------+---------------+
|HOUSING           |NULL           |
|TRAVELLING        |NULL           |
|BOOK STORES       |NULL           |
|AGRICULTURE       |NULL           |
|GOLD LOAN         |NULL           |
|EDUCATIONAL LOAN  |NULL           |
|AUTOMOBILE        |NULL           |
|BUSINESS          |NULL           |
|COMPUTER SOFTWARES|NULL           |
|DINNING           |NULL           |
|SHOPPING          |NULL           |
|RESTAURANTS       |NULL           |
|ELECTRONICS       |NULL           |
|BUILDING          |NULL           |
|RESTAURANT        |NULL           |
|HOME APPLIANCES   |NULL           |
+------------------+---------------+



In [30]:
# Count how many loans are overdue vs not overdue
loan_status_count = df_clean.groupBy("Overdue") \
    .agg(count("*").alias("total_loans")) \
    .orderBy("total_loans", ascending=False)

print("Loans by Overdue status:")
loan_status_count.show(truncate=False)

Loans by Overdue status:
+-------+-----------+
|Overdue|total_loans|
+-------+-----------+
|6      |75         |
|7      |65         |
|1      |63         |
|4      |63         |
|5      |56         |
|8      |48         |
|2      |45         |
|9      |43         |
|3      |42         |
+-------+-----------+



In [31]:
# Top 5 loan categories by average income
top5_categories = df_clean.groupBy("Loan Category") \
    .agg(avg(col("Income")).alias("avg_income")) \
    .orderBy("avg_income", ascending=False) \
    .limit(5)

print("Top 5 Loan Categories by Average Income:")
top5_categories.show(truncate=False)

Top 5 Loan Categories by Average Income:
+------------------+------------------+
|Loan Category     |avg_income        |
+------------------+------------------+
|COMPUTER SOFTWARES|134376.66666666666|
|HOUSING           |74728.19354838709 |
|GOLD LOAN         |70838.31506849315 |
|BUSINESS          |70246.54166666667 |
|BUILDING          |69700.16666666667 |
+------------------+------------------+



**Part 4** — Spark SQL

In [32]:
# Register DataFrame as a temporary SQL view
df_clean.createOrReplaceTempView("loans")

In [33]:
# Loan category with the highest default (Overdue) rate
highest_default_category = spark.sql("""
    SELECT
        `Loan Category`,
        (SUM(CASE WHEN Overdue = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS default_rate
    FROM loans
    GROUP BY `Loan Category`
    ORDER BY default_rate DESC
    LIMIT 1
""")
print("Loan Category with Highest Default Rate:")
highest_default_category.show(truncate=False)


Loan Category with Highest Default Rate:
+-------------+----------------+
|Loan Category|default_rate    |
+-------------+----------------+
|HOUSING      |0.00000000000000|
+-------------+----------------+



In [34]:
# State with most charged-off loans (⚠ No state column in your data)
# For demo, let's assume "Occupation" is a placeholder for a geographic field
most_charged_off = spark.sql("""
    SELECT
        Occupation,
        COUNT(*) AS charged_off_loans
    FROM loans
    WHERE Overdue = 'Yes'
    GROUP BY Occupation
    ORDER BY charged_off_loans DESC
    LIMIT 1
""")
print("Occupation (placeholder for State) with Most Charged-Off Loans:")
most_charged_off.show(truncate=False)

Occupation (placeholder for State) with Most Charged-Off Loans:
+----------+-----------------+
|Occupation|charged_off_loans|
+----------+-----------------+
+----------+-----------------+



**Part 5** — Machine Learning (Optional)

In [35]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:

# Encode categorical columns
loan_category_indexer = StringIndexer(inputCol="Loan Category", outputCol="LoanCategoryIndex")
overdue_indexer = StringIndexer(inputCol="Overdue", outputCol="label")  # Target variable

In [38]:
# Apply indexers
df_indexed = loan_category_indexer.fit(df_clean).transform(df_clean)
df_indexed = overdue_indexer.fit(df_indexed).transform(df_indexed)

In [39]:
# Assemble features (using some numeric columns + encoded category)
feature_cols = ["Age", "Income", "Expenditure", "Family Size", "Loan Amount", "LoanCategoryIndex"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

df_final = assembler.transform(df_indexed)

In [40]:
# Split into train & test
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)

In [45]:
from pyspark.ml.feature import VectorAssembler

# Choose your input feature columns
feature_cols = ["Age", "Income", "Expenditure", "Family Size", "Loan Amount"]

# Create VectorAssembler
assembler = VectorAssembler(
    inputCols=feature_cols,   # Columns to combine
    outputCol="features",     # Name of new features column
    handleInvalid="keep"      # Avoids errors if nulls exist
)

# Apply transformation
df_with_features = assembler.transform(df_clean)

# Check results
df_with_features.select(feature_cols + ["features"]).show(5, truncate=False)


+---+------+-----------+-----------+-----------+------------------------------+
|Age|Income|Expenditure|Family Size|Loan Amount|features                      |
+---+------+-----------+-----------+-----------+------------------------------+
|30 |53957 |34709      |2          |NULL       |[30.0,53957.0,34709.0,2.0,NaN]|
|39 |77699 |12482      |4          |NULL       |[39.0,77699.0,12482.0,4.0,NaN]|
|51 |84966 |42591      |3          |NULL       |[51.0,84966.0,42591.0,3.0,NaN]|
|56 |30000 |15426      |5          |NULL       |[56.0,30000.0,15426.0,5.0,NaN]|
|55 |34999 |19888      |6          |NULL       |[55.0,34999.0,19888.0,6.0,NaN]|
+---+------+-----------+-----------+-----------+------------------------------+
only showing top 5 rows

